## Pre-trained ConvNet with for the Dogs vs Cats dataset
The dataset can be downloaded at [Kaggle](https://www.kaggle.com/c/dogs-vs-cats/data).
We use the pre-trained ConvNet to achieve the bottleneck features (the features generated from block5_pool layer, which will be pass through the following fully-connected layers)
> Bottleneck features are generated from a multi-layer perceptron in which one of the internal layers has a small number of hidden units, relative to the size of the other layers according to the following paper. [Reference](https://www.microsoft.com/en-us/research/publication/improved-bottleneck-features-using-pretrained-deep-neural-networks/?from=http%3A%2F%2Fresearch.microsoft.com%2Fpubs%2F153642%2Fbottleneck-interspeech2011-pub.pdf)
>

The following codes are modified based on [Reference](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html) created by Francois Chollet.

**Remark** Although Keras and tf.keras in most cases are compatible, but sometimes you will have unexpected errors. Most of the suggestions on the website are for Keras, some of them will not work for tf.keras.

In [7]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/dogs-vs-cats-small/train'
validation_data_dir = 'data/dogs-vs-cats-small/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16


def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # load the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    
    # pre-processing pipeline
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    # generate the bottleneck train features and save them as the new training data
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)
   
    # generate the bottleneck validation features and save them as the new validation data
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),bottleneck_features_validation)


def train_top_model():
    # load the new training and validation data
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))
    
    # build a fully connect network
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottleneck_features()
train_top_model()

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 1s 405us/step - loss: 0.6748 - acc: 0.7575 - val_loss: 0.2816 - val_acc: 0.8825
Epoch 2/50
2000/2000 [==============================] - 0s 204us/step - loss: 0.3732 - acc: 0.8525 - val_loss: 0.4034 - val_acc: 0.8450
Epoch 3/50
2000/2000 [==============================] - 0s 207us/step - loss: 0.3359 - acc: 0.8715 - val_loss: 0.3509 - val_acc: 0.8488
Epoch 4/50
2000/2000 [==============================] - 0s 215us/step - loss: 0.2785 - acc: 0.8890 - val_loss: 0.2806 - val_acc: 0.8912
Epoch 5/50
2000/2000 [==============================] - 0s 202us/step - loss: 0.2472 - acc: 0.9110 - val_loss: 0.2743 - val_acc: 0.9012
Epoch 6/50
2000/2000 [==============================] - 0s 203us/step - loss: 0.2174 - acc: 0.9165 - val_loss: 0.3375 - val_acc: 0.8925
Epoch 7/50
2000/2000 [==============================] 

In [8]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/dogs-vs-cats-small/train'
validation_data_dir = 'data/dogs-vs-cats-small/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16


def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottleneck_features()
train_top_model()

Found 2000 images belonging to 2 classes.


RuntimeError: You must compile your model before using it.

Unexpected error, compile the model before using it even for the pre-trained model?

In [12]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

1.12.0
2.2.4
